In [1]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import cv2 as cv
import numpy as np
# import SciPy as scipy

2023-01-20 21:37:01.258495: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-20 21:37:01.445602: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-20 21:37:01.445622: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-20 21:37:02.477055: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
img_size = (224, 224)

train_path = './Assignment-3/dataset/training/'
test_path = './Assignment-3/dataset/testing/'
val_path = './Assignment-3/dataset/validation/'

In [3]:
x_train = []
for folder in os.listdir(train_path):
    sub_path = train_path + '/' + folder
    for img in os.listdir(sub_path):
        img_path = sub_path + '/' + img
        img = cv.imread(img_path, 1)
        img = cv.resize(img, img_size)
#         print(img.shape)
#         break
        x_train.append(img)
# img.shape

In [4]:
x_test = []
for folder in os.listdir(test_path):
    sub_path = train_path + '/' + folder
    for img in os.listdir(sub_path):
        img_path = sub_path + '/' + img
        img = cv.imread(img_path, 1)
        img = cv.resize(img, img_size)
        x_test.append(img)

In [5]:
x_val = []
for folder in os.listdir(val_path):
    sub_path = train_path + '/' + folder
    for img in os.listdir(sub_path):
        img_path = sub_path + '/' + img
        img = cv.imread(img_path, 1)
        img = cv.resize(img, img_size)
        x_val.append(img)

In [6]:
train_x = np.array(x_train)
test_x = np.array(x_test)
val_x = np.array(x_val)

In [7]:
train_x = train_x / 255.0
test_x = test_x / 255.0
val_x = val_x / 255.0

In [8]:
train_datagen = ImageDataGenerator(rescale = 1.0 / 255)
test_datagen = ImageDataGenerator(rescale = 1.0 / 255)
val_dtagen = ImageDataGenerator(rescale = 1.0 / 255)

In [9]:
train_set = train_datagen.flow_from_directory(train_path, target_size = img_size, batch_size = 32, class_mode = 'binary')
test_set = train_datagen.flow_from_directory(test_path, target_size = img_size, batch_size = 32, class_mode = 'binary')
val_set = train_datagen.flow_from_directory(val_path, target_size = img_size, batch_size = 32, class_mode = 'binary')

Found 468 images belonging to 2 classes.
Found 26 images belonging to 2 classes.
Found 26 images belonging to 2 classes.


In [10]:
train_y = train_set.classes
test_y = test_set.classes
val_y = val_set.classes

In [18]:
model = Sequential()

model.add(Conv2D(input_shape=(224, 224, 3), filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# model.build(input_shape = (100, 224, 224, 1))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 222, 222, 64)      1792      
                                                                 
 conv2d_3 (Conv2D)           (None, 220, 220, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 110, 110, 64)     0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 774400)            0         
                                                                 
 dense_3 (Dense)             (None, 16)                12390416  
                                                                 
 dense_4 (Dense)             (None, 32)                544       
                                                      

In [20]:
model.compile(
  loss='binary_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)

In [13]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)
#Early stopping to avoid overfitting of model

In [16]:
# fit the model
history=model.fit(
    train_x,
    train_y,
    validation_data=(test_x,test_y),
    epochs=30,
#     callbacks=[early_stop],
    shuffle=True)

Epoch 1/30
15/15 [==============================] - ETA: 0s - loss: 0.6975 - accuracy: 0.4829

ValueError: Data cardinality is ambiguous:
  x sizes: 468
  y sizes: 26
Make sure all arrays contain the same number of samples.

In [23]:
history = model.fit(
    train_set,
#     samples_per_epoch=234,
    epochs=10,
    validation_data=val_set #,
#     nb_val_samples=13
    )

Epoch 1/10
15/15 [==============================] - 110s 7s/step - loss: 3.2551e-07 - accuracy: 1.0000 - val_loss: 3.6393e-07 - val_accuracy: 1.0000
Epoch 2/10
15/15 [==============================] - 112s 7s/step - loss: 3.1144e-07 - accuracy: 1.0000 - val_loss: 3.4846e-07 - val_accuracy: 1.0000
Epoch 3/10
15/15 [==============================] - 107s 7s/step - loss: 2.9766e-07 - accuracy: 1.0000 - val_loss: 3.3473e-07 - val_accuracy: 1.0000
Epoch 4/10
15/15 [==============================] - 108s 7s/step - loss: 2.8480e-07 - accuracy: 1.0000 - val_loss: 3.2009e-07 - val_accuracy: 1.0000
Epoch 5/10
15/15 [==============================] - 111s 7s/step - loss: 2.7258e-07 - accuracy: 1.0000 - val_loss: 3.0622e-07 - val_accuracy: 1.0000
Epoch 6/10
15/15 [==============================] - 111s 7s/step - loss: 2.6012e-07 - accuracy: 1.0000 - val_loss: 2.9395e-07 - val_accuracy: 1.0000
Epoch 7/10
15/15 [==============================] - 117s 8s/step - loss: 2.4866e-07 - accuracy: 1.0000 - v

In [24]:
model.evaluate(test_set)

1/1 [==============================] - 4s 4s/step - loss: 2.3498e-07 - accuracy: 1.0000


[2.3498419920997549e-07, 1.0]